In [1]:
import csv
import os
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf
import matplotlib.pyplot as plt
from pywt import wavedec

2023-09-28 20:08:02.272865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
# import os
# dataset_path = '../Preprocessing/db6'
# for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#     for f in filenames:
#         file_path = os.path.join(dirpath, f)
#         print(file_path)


../Preprocessing/db6/data_mfccDWTcqtdb6L8.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L7.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L6.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L4.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L3.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L1.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L5.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L9.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L2.csv
../Preprocessing/db6/data_mfccDWTcqtdb6L10.csv


In [22]:
# header = 'filename'
# header += ' hasil'
# header = header.split()

# file = open('../Preprocessing/hasil.csv', 'w', newline='')
# with file:
#   writer = csv.writer(file)
#   writer.writerow(header)

In [4]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          def build_model():
            model = models.Sequential(
                [
                    layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
                    layers.MaxPooling1D((3), strides=(2), padding="same"),

                    layers.Conv1D(32, (3) , activation="relu"),
                    layers.MaxPooling1D((3), strides=(2), padding="same"),

                    layers.Flatten(),
                    layers.Dense(32, activation="relu"),
                    layers.Dense(5, activation="softmax"),
                ]
            )

            return model

          model = build_model()


          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])
          history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))
          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/db1/data_mfccRMScqtdb1L3.csv
Epoch 1/50
13/13 [==============================] - 2s 62ms/step - loss: 3.0224 - acc: 0.0662 - val_loss: 2.2648 - val_acc: 0.1850
Epoch 2/50
13/13 [==============================] - 0s 28ms/step - loss: 2.1828 - acc: 0.2937 - val_loss: 1.7676 - val_acc: 0.3750
Epoch 3/50
13/13 [==============================] - 0s 34ms/step - loss: 1.7260 - acc: 0.4075 - val_loss: 1.4728 - val_acc: 0.4850
Epoch 4/50
13/13 [==============================] - 0s 27ms/step - loss: 1.4397 - acc: 0.4762 - val_loss: 1.2997 - val_acc: 0.5250
Epoch 5/50
13/13 [==============================] - 0s 23ms/step - loss: 1.2557 - acc: 0.5025 - val_loss: 1.1688 - val_acc: 0.5350
Epoch 6/50
13/13 [==============================] - 0s 34ms/step - loss: 1.1159 - acc: 0.5750 - val_loss: 1.0580 - val_acc: 0.6750
Epoch 7/50
13/13 [==============================] - 0s 38ms/step - loss: 1.0024 - acc: 0.6850 - val_loss: 0.9673 - val_acc: 0.7100
Epoch 8/50
13/13 [=

In [23]:
# dataset_path = '../Preprocessing/db10'
# for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#   for f in filenames:
#     file_path = os.path.join(dirpath, f)
#     print('Processing: ', file_path)
#     data = pd.read_csv(file_path)
#     # data.head()

#     X = data.drop(['label', 'filename'], axis=1)
#     y = data.iloc[:, -1]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     lb = LabelEncoder()
#     X_train = np.expand_dims(X_train, axis=2)
#     X_test = np.expand_dims(X_test, axis=2)
#     y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
#     y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

#     X_train = X_train[..., np.newaxis]
#     X_test = X_test[..., np.newaxis]

#     def build_model():
#         model = models.Sequential(
#             [
#                 layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
#                 layers.MaxPooling1D((3), strides=(2), padding="same"),

#                 layers.Conv1D(32, (3) , activation="relu"),
#                 layers.MaxPooling1D((3), strides=(2), padding="same"),

#                 layers.Flatten(),
#                 layers.Dense(64, activation="relu"),
#                 layers.Dense(5, activation="softmax"),
#             ]
#         )

#         return model

#     model = build_model()


#     optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#     model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])
#     history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


#     # evaluate model on test set
#     test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
#     print("Accuracy on test set is: {}".format(test_accuracy))

#     to_append = f'{file_path}'
#     to_append += f' {test_accuracy}'
#     file = open('../Preprocessing/hasil.csv', 'a', newline='')
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())


Processing:  ../Preprocessing/db10/data_mfccDWTcqtdb10L5.csv


Epoch 1/50
25/25 [==============================] - 4s 46ms/step - loss: 1.9350 - acc: 0.2163 - val_loss: 1.5532 - val_acc: 0.3350
Epoch 2/50
25/25 [==============================] - 1s 29ms/step - loss: 1.4351 - acc: 0.4663 - val_loss: 1.2932 - val_acc: 0.5400
Epoch 3/50
25/25 [==============================] - 1s 28ms/step - loss: 1.2171 - acc: 0.5650 - val_loss: 1.1236 - val_acc: 0.6000
Epoch 4/50
25/25 [==============================] - 1s 31ms/step - loss: 1.0717 - acc: 0.6400 - val_loss: 0.9926 - val_acc: 0.6800
Epoch 5/50
25/25 [==============================] - 1s 24ms/step - loss: 0.9577 - acc: 0.6800 - val_loss: 0.8850 - val_acc: 0.7500
Epoch 6/50
25/25 [==============================] - 1s 26ms/step - loss: 0.8616 - acc: 0.7350 - val_loss: 0.7883 - val_acc: 0.8100
Epoch 7/50
25/25 [==============================] - 1s 25ms/step - loss: 0.7854 - acc: 0.7563 - val_loss: 0.7026 - val_acc: 0.8400
Epoch 8/50
25/25 [==============================] - 0s 15ms/step - loss: 0.7210 - a